In [ ]:
from planning_through_contact.geometry.collision_geometry.box_2d import Box2d
from planning_through_contact.geometry.rigid_body import RigidBody
from planning_through_contact.planning.planar.planar_plan_config import ContactCostType, PlanarPlanConfig, SliderPusherSystemConfig
from planning_through_contact.geometry.collision_geometry.collision_geometry import ContactLocation, PolytopeContactLocation
from planning_through_contact.geometry.planar.face_contact import FaceContactMode
from planning_through_contact.geometry.planar.planar_pose import PlanarPose
from planning_through_contact.geometry.planar.planar_pushing_trajectory import PlanarPushingTrajectory
from planning_through_contact.geometry.planar.trajectory_builder import PlanarTrajectoryBuilder
from planning_through_contact.visualize.planar_pushing import visualize_planar_pushing_trajectory
from planning_through_contact.planning.planar.planar_plan_config import PlanarPushingStartAndGoal
from planning_through_contact.visualize.analysis import analyze_mode_result
from planning_through_contact.planning.planar.planar_plan_config import ContactCost
from planning_through_contact.planning.planar.planar_plan_config import ContactConfig
from planning_through_contact.visualize.analysis import analyze_plan
from planning_through_contact.visualize.planar_pushing import compare_trajs
from planning_through_contact.planning.planar.planar_plan_config import NonCollisionCost, PlanarSolverParams

from pydrake.solvers import MosekSolver
from pydrake.solvers import ClarabelSolver
from pydrake.solvers import SnoptSolver, SolverOptions
from IPython.display import HTML, SVG, display

clarabel = ClarabelSolver() # Clarabel seems to be ~10x slower than Mosek
mosek = MosekSolver()
snopt = SnoptSolver()

In [ ]:
# Define slider
from planning_through_contact.geometry.collision_geometry.t_pusher_2d import TPusher2d


mass = 0.3
tee_geometry = TPusher2d()
tee = RigidBody("tee", tee_geometry, mass)

# Define pusher
pusher_radius = 0.035

# Define slider-pusher system
cfg = SliderPusherSystemConfig(
    slider=tee,
    pusher_radius=pusher_radius,
    friction_coeff_slider_pusher=0.4,
    friction_coeff_table_slider=0.5,
    integration_constant=0.4
)

# Define contact cost
contact_cost = ContactCost(
    cost_type=ContactCostType.STANDARD,
    keypoint_arc_length=0.1,
    linear_arc_length=None,
    angular_arc_length=None,
    force_regularization=0.3,
    keypoint_velocity_regularization=None,
    ang_velocity_regularization=1.0,
    lin_velocity_regularization=0.5,
    trace=None,
    mode_transition_cost=0.1,
)
contact_config = ContactConfig(contact_cost)

# Define non-collision cost
non_collision_cost = NonCollisionCost(
    distance_to_object_quadratic=None,
    distance_to_object_quadratic_preferred_distance=None,
    distance_to_object_socp=None,
    pusher_velocity_regularization=0.005,
    pusher_arc_length=0.005,
)

plan_cfg = PlanarPlanConfig(
    dynamics_config=cfg,
    num_knot_points_contact=3,
    num_knot_points_non_collision=3,
    use_band_sparsity=True,
    contact_config=contact_config,
    non_collision_cost=non_collision_cost,
    continuity_on_pusher_velocity=True,
    allow_teleportation=False,
)

solver_params = PlanarSolverParams(
    measure_solve_time=False,
    gcs_max_rounded_paths=20,
    print_flows=False,
    solver="mosek",
    print_solver_output=False,
    save_solver_output=False,
    print_path=False,
    print_cost=False,
    assert_result=False,
)

In [ ]:
import time

from planning_through_contact.planning.planar.planar_pushing_planner import PlanarPushingPlanner


def plan(plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose):
    plan_cfg.start_and_goal = PlanarPushingStartAndGoal(
        slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
    )

    planner = PlanarPushingPlanner(plan_cfg)
    planner.formulate_problem()

    start_time = time.time()
    solver_params.gcs_max_rounded_paths = 0
    lb_result = planner._solve(solver_params)
    #print(f"Lower bound elapsed time: {time.time() - start_time}")
    lb_cost = lb_result.get_optimal_cost()
    print(f"Cost lower bound: {lb_cost}")

    solver_params.gcs_max_rounded_paths = 20
    start_time = time.time()
    plan = planner.plan_path(solver_params)
    print(f"SDP elapsed time: {time.time() - start_time}")
    traj = plan.to_traj()
    ani = visualize_planar_pushing_trajectory(traj, visualize_knot_points=True)

    cost = planner.path.result.get_optimal_cost()
    print(f"SDP cost: {cost}")

    start_time = time.time()
    traj_rounded = plan.to_traj(do_rounding=True, solver_params=solver_params)
    print(f"Nonlinear rounding elapsed time: {time.time() - start_time}")

    rounded_cost = planner.path.rounded_result.get_optimal_cost()
    print(f"Rounded cost: {rounded_cost}")

    ani_rounded = visualize_planar_pushing_trajectory(traj_rounded, visualize_knot_points=True)
    print(f"Local optimality gap: {(cost / rounded_cost):.2%}")
    print(f"Global optimality gap: {(lb_cost / rounded_cost):.2%}")
    
    return traj, ani, traj_rounded, ani_rounded, planner.path

In [ ]:
skip_easy_trajs = False

In [ ]:
if not skip_easy_trajs:
    slider_initial_pose = PlanarPose(-0.3, 0.05, 0.2)
    slider_final_pose = PlanarPose(0.0, 0.0, 0.0)
    pusher_initial_pose = PlanarPose(-0.5, 0.0, 0.0)
    pusher_final_pose = PlanarPose(-0.5, 0.0, 0.0)
    traj, ani, traj_rounded, ani_rounded, path = plan(
        plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
    )
    
    print("RELAXED SOLUTION:")
    display(HTML(ani.to_jshtml()))
    analyze_plan(path)

    print("ROUNDED SOLUTION:")
    display(HTML(ani_rounded.to_jshtml()))
    analyze_plan(path, rounded=True)

    print("COMPARING TRAJECTORIES:")
    compare_trajs(traj, traj_rounded, traj_a_legend="relaxed", traj_b_legend="rounded")

In [ ]:
if not skip_easy_trajs:
    slider_initial_pose = PlanarPose(0.0, 0.0, 0.0)
    slider_final_pose = PlanarPose(0.0, -0.2, 0.3)
    pusher_initial_pose = PlanarPose(-0.5, 0.0, 0.0)
    pusher_final_pose = PlanarPose(-0.5, 0.0, 0.0)
    traj, ani, traj_rounded, ani_rounded, path = plan(
        plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
    )
    
    print("RELAXED SOLUTION:")
    display(HTML(ani.to_jshtml()))
    analyze_plan(path)

    print("ROUNDED SOLUTION:")
    display(HTML(ani_rounded.to_jshtml()))
    analyze_plan(path, rounded=True)

    print("COMPARING TRAJECTORIES:")
    compare_trajs(traj, traj_rounded, traj_a_legend="relaxed", traj_b_legend="rounded")

In [ ]:
if not skip_easy_trajs:
    slider_initial_pose = PlanarPose(0.2, 0.0, 0.4)
    slider_final_pose = PlanarPose(0.0, 0.0, 0.0)
    pusher_initial_pose = PlanarPose(-0.5, 0.0, 0.0)
    pusher_final_pose = PlanarPose(-0.5, 0.0, 0.0)
    traj, ani, traj_rounded, ani_rounded, path = plan(
        plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
    )
    
    print("RELAXED SOLUTION:")
    display(HTML(ani.to_jshtml()))
    analyze_plan(path)

    print("ROUNDED SOLUTION:")
    display(HTML(ani_rounded.to_jshtml()))
    analyze_plan(path, rounded=True)

    print("COMPARING TRAJECTORIES:")
    compare_trajs(traj, traj_rounded, traj_a_legend="relaxed", traj_b_legend="rounded")

In [ ]:
if not skip_easy_trajs:
    slider_initial_pose = PlanarPose(-0.01, -0.2, 0.4)
    slider_final_pose = PlanarPose(0.0, 0.0, 0.0)
    pusher_initial_pose = PlanarPose(-0.5, 0.0, 0.0)
    pusher_final_pose = PlanarPose(-0.5, 0.0, 0.0)
    traj, ani, traj_rounded, ani_rounded, path = plan(
        plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
    )
    
    print("RELAXED SOLUTION:")
    display(HTML(ani.to_jshtml()))
    analyze_plan(path)

    print("ROUNDED SOLUTION:")
    display(HTML(ani_rounded.to_jshtml()))
    analyze_plan(path, rounded=True)

    print("COMPARING TRAJECTORIES:")
    compare_trajs(traj, traj_rounded, traj_a_legend="relaxed", traj_b_legend="rounded")

In [ ]:
slider_initial_pose = PlanarPose(-0.01, -0.2, 2.4)
slider_final_pose = PlanarPose(0.0, 0.0, 0.0)
pusher_initial_pose = PlanarPose(-0.5, 0.0, 0.0)
pusher_final_pose = PlanarPose(-0.5, 0.0, 0.0)
traj, ani, traj_rounded, ani_rounded, path = plan(
    plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
)

print("RELAXED SOLUTION:")
display(HTML(ani.to_jshtml()))
analyze_plan(path)

print("ROUNDED SOLUTION:")
display(HTML(ani_rounded.to_jshtml()))
analyze_plan(path, rounded=True)

print("COMPARING TRAJECTORIES:")
compare_trajs(traj, traj_rounded, traj_a_legend="relaxed", traj_b_legend="rounded")

In [ ]:
slider_initial_pose = PlanarPose(-0.01, -0.2, 1.4)
slider_final_pose = PlanarPose(0.0, 0.0, 0.0)
pusher_initial_pose = PlanarPose(-0.5, 0.0, 0.0)
pusher_final_pose = PlanarPose(-0.5, 0.0, 0.0)
traj, ani, traj_rounded, ani_rounded, path = plan(
    plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
)

print("RELAXED SOLUTION:")
display(HTML(ani.to_jshtml()))
analyze_plan(path)

print("ROUNDED SOLUTION:")
display(HTML(ani_rounded.to_jshtml()))
analyze_plan(path, rounded=True)

print("COMPARING TRAJECTORIES:")
compare_trajs(traj, traj_rounded, traj_a_legend="relaxed", traj_b_legend="rounded")

In [ ]:
slider_initial_pose = PlanarPose(0.3, 0.2, -2.4)
slider_final_pose = PlanarPose(0.0, 0.0, 0.0)
pusher_initial_pose = PlanarPose(-0.5, 0.0, 0.0)
pusher_final_pose = PlanarPose(-0.5, 0.0, 0.0)
traj, ani, traj_rounded, ani_rounded, path = plan(
    plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
)
  
print("RELAXED SOLUTION:")
display(HTML(ani.to_jshtml()))
analyze_plan(path)

print("ROUNDED SOLUTION:")
display(HTML(ani_rounded.to_jshtml()))
analyze_plan(path, rounded=True)

print("COMPARING TRAJECTORIES:")
compare_trajs(traj, traj_rounded, traj_a_legend="relaxed", traj_b_legend="rounded")

In [ ]:
slider_initial_pose = PlanarPose(0.3, -0.3, 3.14159/2)
slider_final_pose = PlanarPose(0.0, 0.0, 0.0)
pusher_initial_pose = PlanarPose(-0.5, 0.0, 0.0)
pusher_final_pose = PlanarPose(-0.5, 0.0, 0.0)
traj, ani, traj_rounded, ani_rounded, path = plan(
    plan_cfg, slider_initial_pose, slider_final_pose, pusher_initial_pose, pusher_final_pose
)
  
print("RELAXED SOLUTION:")
display(HTML(ani.to_jshtml()))
analyze_plan(path)

print("ROUNDED SOLUTION:")
display(HTML(ani_rounded.to_jshtml()))
analyze_plan(path, rounded=True)

print("COMPARING TRAJECTORIES:")
compare_trajs(traj, traj_rounded, traj_a_legend="relaxed", traj_b_legend="rounded")